In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [4]:
# 将训练集按照 6:4 的比例进行切割，从而最终我们将得到 15,000
# 个训练样本, 10,000 个验证样本以及 25,000 个测试样本
(train_data, validation_data, test_data) = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [5]:
train_examples_batch,train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.',
       b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>

In [7]:
embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding,input_shape=[],
                          dtype=tf.string,trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 2.209591  , -2.7093675 ,  3.6802928 , -1.0291991 , -4.1671185 ,
        -2.4566064 , -2.2519937 , -0.36589956,  1.9485804 , -3.1104462 ,
        -2.4610963 ,  1.3139242 , -0.9161584 , -0.16625322, -3.723651  ,
         1.8498232 ,  3.499562  , -1.2373022 , -2.8403084 , -1.213074  ],
       [ 1.9055302 , -4.11395   ,  3.6038654 ,  0.28555924, -4.658998  ,
        -5.5433393 , -3.2735848 ,  1.9235417 ,  3.8461034 ,  1.5882455 ,
        -2.64167   ,  0.76057523, -0.14820506,  0.9115291 , -6.45758   ,
         2.3990374 ,  5.0985413 , -3.2776263 , -3.2652326 , -1.2345369 ],
       [ 3.6510668 , -4.7066135 ,  4.71003   , -1.7002777 , -3.7708545 ,
        -3.709126  , -4.222776  ,  1.946586  ,  6.1182513 , -2.7392752 ,
        -5.4384456 ,  2.7078724 , -2.1263676 , -0.7084146 , -5.893995  ,
         3.1602864 ,  3.8389287 , -3.318196  , -5.1542974 , -2.4051712 ]],
      dtype=float32)>

In [14]:
train_examples_batch[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'This is a big step down after the surprisingly enjoyable original. This sequel isn\'t nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is toned down in this entry. The performances are all adequate, but this time the script lets us down. The action is merely routine and the plot is only mildly interesting, so I need lots of silly laughs in order to stay entertained during a "Trancers" movie. Unfortunately, the laughs are few and far between, and so, this film is watchable at best.'>

In [15]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [17]:
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=10,
                   validation_data = validation_data.batch(512),
                   verbose=1)

Epoch 1/10
30/30 [==============================].9851 - accuracy: 0.43 - 2s 769ms/step - loss: 0.9482 - accuracy: 0.452 - 2s 544ms/step - loss: 0.9236 - accuracy: 0.455 - 2s 429ms/step - loss: 0.9043 - accuracy: 0.466 - 2s 358ms/step - loss: 0.8790 - accuracy: 0.481 - 2s 313ms/step - loss: 0.8589 - accuracy: 0.488 - 2s 279ms/step - loss: 0.8437 - accuracy: 0.497 - 2s 257ms/step - loss: 0.8408 - accuracy: 0.497 - 2s 238ms/step - loss: 0.8337 - accuracy: 0.503 - 2s 222ms/step - loss: 0.8320 - accuracy: 0.502 - 2s 210ms/step - loss: 0.8285 - accuracy: 0.503 - 2s 200ms/step - loss: 0.8224 - accuracy: 0.510 - 2s 192ms/step - loss: 0.8172 - accuracy: 0.514 - 3s 184ms/step - loss: 0.8148 - accuracy: 0.517 - 3s 178ms/step - loss: 0.8104 - accuracy: 0.521 - 3s 172ms/step - loss: 0.8051 - accuracy: 0.524 - 3s 167ms/step - loss: 0.8016 - accuracy: 0.527 - 3s 162ms/step - loss: 0.7989 - accuracy: 0.528 - 3s 158ms/step - loss: 0.7974 - accuracy: 0.530 - 3s 154ms/step - loss: 0.7942 - accuracy: 0.5

30/30 [==============================] - ETA: 12s - loss: 0.5077 - accuracy: 0.757 - ETA: 7s - loss: 0.5012 - accuracy: 0.749 - ETA: 5s - loss: 0.4929 - accuracy: 0.76 - ETA: 4s - loss: 0.4887 - accuracy: 0.76 - ETA: 3s - loss: 0.4889 - accuracy: 0.76 - ETA: 3s - loss: 0.4858 - accuracy: 0.76 - ETA: 3s - loss: 0.4857 - accuracy: 0.76 - ETA: 2s - loss: 0.4867 - accuracy: 0.76 - ETA: 2s - loss: 0.4828 - accuracy: 0.77 - ETA: 2s - loss: 0.4838 - accuracy: 0.77 - ETA: 2s - loss: 0.4843 - accuracy: 0.76 - ETA: 2s - loss: 0.4855 - accuracy: 0.76 - ETA: 1s - loss: 0.4864 - accuracy: 0.76 - ETA: 1s - loss: 0.4855 - accuracy: 0.76 - ETA: 1s - loss: 0.4846 - accuracy: 0.76 - ETA: 1s - loss: 0.4825 - accuracy: 0.77 - ETA: 1s - loss: 0.4805 - accuracy: 0.77 - ETA: 1s - loss: 0.4812 - accuracy: 0.77 - ETA: 1s - loss: 0.4821 - accuracy: 0.77 - ETA: 1s - loss: 0.4822 - accuracy: 0.77 - ETA: 0s - loss: 0.4819 - accuracy: 0.77 - ETA: 0s - loss: 0.4811 - accuracy: 0.77 - ETA: 0s - loss: 0.4819 - accurac

In [18]:
results = model.evaluate(test_data.batch(512),verbose=2)
for name,value in zip(model.metrics_names,results):
    print("%s: %.3f" % (name, value))

loss: 0.407
accuracy: 0.818
